In [1]:
library(HMM)
library(mclust)

Warning message:
"package 'mclust' was built under R version 3.6.3"Package 'mclust' version 5.4.7
Type 'citation("mclust")' for citing this R package in publications.


In [2]:
dim(faithful)
head(faithful)

[1] 272   2

eruptions,waiting
3.600,79
1.800,54
3.333,74
2.283,62
4.533,85
2.883,55


In [3]:
mygauss <- function(data, mean, Sigma){
    p = nrow(Sigma)
    svd_obj = svd(Sigma)
    D = svd_obj$d
    U = svd_obj$u
    V = svd_obj$v
    Dt = diag(1/sqrt(D))
    A = t(data)
    Sinv = U %*% Dt^2 %*% t(U) 

    #xt = Dt %*% t(U) %*% data
    #mt = Dt %*% t(U) %*% mean
    #a = t(xt-mt) %*% Sinv %*% (xt-mt)
    a = (A - mean) * (Sinv %*% (A - mean))
    a = colSums(a)
    return (exp(-0.5 * a)/((2*pi)^p * (det(Sigma))^0.5))
}

Estep <- function(data, G, para){
    # Your Code
    prob <- para$prob
    mu <- para$mean
    Sigma <- para$Sigma
    loglik <- para$loglik
    n <- nrow(data)
    # need to come up with a way to evalue multivariate gaussian distrbutions,
    # but let's focus on getting the algorithm correct right now.
    # assume we have G means, G Sigmas, G pis
    p = matrix(0, n, G)
    for (k in 1:G){
        p[,k] = prob[k] * mygauss(data, mu[,k], Sigma)
    }
    #p = prob * mygauss(data, mu, Sigma)
    s = rowSums(p)
    p = p / s
  
    # Return the n-by-G probability matrix
    return(p)
  }

Mstep <- function(data, G, para, post.prob){ 
    # Your Code
    # Return the updated parameters
    para$prob = colSums(post.prob) / nrow(post.prob)
    mu = t(data) %*% post.prob
    #mu_sum = colSums(mu)
    mu_sum = colSums(post.prob)
    n = nrow(mu)
    for (k in 1:n){
        mu[k,] = mu[k,] / mu_sum
    }
    para$mean = mu
    Sigma = matrix(0, n, n)
    # This loop is giving the first step Sigma very close agreement with the 
    # R method, but not exact agreement. 
    for (k in 1:G) {
        x = t(data) - mu[,k]
        P = diag(post.prob[,k])
        Sigma = Sigma + x %*% P %*% t(x)   
    }
    
    para$Sigma = Sigma / (colSums(post.prob) * n)
    
    return (para)
  }

loglik <- function(data, G, para){
    # compute loglikelihood
    ll = para$loglik
    
    return (ll)
}

myEM <- function(data, itmax, G, para){
  # itmax: number of of iterations
  # G:     number of components
  # para:  list of (prob, mean, Sigma, loglik)
  d = as.matrix(data)
  for(t in 1:itmax){
    print(t)
    #print(para$Sigma)
    post.prob <- Estep(d, G, para)
    #print(post.prob)
    para <- Mstep(d, G, para, post.prob)
  }
  
  # update para$loglik   
  #para[4] = loglik(data, G, para)
  
  return(para)
}

In [4]:
n <- nrow(faithful)
G <- 2
set.seed(7568)  # replace 234 by the last 4-dig of your University ID
gID <- sample(1:G, n, replace = TRUE)
Z <- matrix(0, n, G)
for(k in 1:G)
  Z[gID == k, k] <- 1 
ini0 <- mstep(modelName="EEE", faithful , Z)$parameters

In [ ]:
para0 <- list(prob = ini0$pro, 
              mean = ini0$mean,
              Sigma = ini0$variance$Sigma, 
              loglik = NULL)

In [ ]:
myEM(d=faithful, itmax=20, G=G, para=para0)

In [ ]:
# Test E step
para_test <- list(pro = ini0$pro,
                  mean = ini0$mean,
                  variance = ini0$variance,
                  Vinv = NULL)
estep(data=faithful, modelName="EEE", parameters=para_test)$z

In [ ]:
d = as.matrix(faithful)
G = 2
post.prob <- Estep(d, G, para0)
post.prob

In [ ]:
# Test MStep
testm_res <- mstep(data=faithful, modelName="EEE", z = post.prob)$parameters

In [ ]:
para <- Mstep(d, G, para0, post.prob)
para

In [ ]:
testm_res$variance$Sigma

# Problem 2

In [5]:
myBW = function(x, para, n.iter = 100){
  # Input:
  # x: T-by-1 observation sequence
  # para: initial parameter value
  # Output updated para value (A and B; we do not update w)
  
  for(i in 1:n.iter){
    para = BW.onestep(x, para)
  }
  return(para)
}

In [63]:
BW.onestep = function(x, para){
    # Input: 
    # x: T-by-1 observation sequence
    # para: mx, mz, and current para values for
    #    A: initial estimate for mz-by-mz transition matrix
    #    B: initial estimate for mz-by-mx emission matrix
    #    w: initial estimate for mz-by-1 initial distribution over Z_1
    # Output the updated parameters after one iteration
    # We DO NOT update the initial distribution w

    T = length(x)
    mz = para$mz
    mx = para$mx
    A = para$A
    B = para$B
    w = para$w
    alp = forward.prob(x, para)
    beta = backward.prob(x, para)

    myGamma = array(0, dim=c(mz, mz, T-1))
    myGamma_i = matrix(0, T, mz)
    #######################################
    ## YOUR CODE: 
    ## Compute gamma_t(i,j) P(Z[t] = i, Z[t+1]=j), 
    for(t in 1:T-1){
        myGamma[,,t] = alp[t,] %*% A * B[, x[t+1]] * beta[t+1, ]
    }

    myGamma_i[1:(T-1),] = t(colSums(myGamma))
    myGamma_i[T,] = rowSums(myGamma)

    ## which are stored in an array, myGamma
    #######################################

    # M-step for parameter A
    #######################################
    ## YOUR CODE: 
    
    t_sums = as.matrix(apply(myGamma, MARGIN=c(1,2), sum))
    denom = colSums(t_sums)
    A = t_sums/denom
    
    #######################################

    # M-step for parameter B
    #######################################
    ## YOUR CODE: 


    B[] = 0
    for (i in 1:mz) {
        for (l in 1:mx) {
        idx = which(x[1:T] == l)                      
        B[i,l] = sum(myGamma_i[idx,i]) / sum(myGamma_i[,i])
        }
    }
    
    #######################################

    para$A = A
    para$B = B
    return(para)
}

In [64]:
data = scan("coding4_part2_data.txt")

mz = 2
mx = 3
ini.w = rep(1, mz); ini.w = ini.w / sum(ini.w)
ini.A = matrix(1, 2, 2); ini.A = ini.A / rowSums(ini.A)
ini.B = matrix(1:6, 2, 3); ini.B = ini.B / rowSums(ini.B)
ini.para = list(mz = 2, mx = 3, w = ini.w,
                A = ini.A, B = ini.B)

In [65]:
myout = myBW(data, ini.para, n.iter = 100)

In [8]:
forward.prob = function(x, para){
  # Output the forward probability matrix alp 
  # alp: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  alp = matrix(0, T, mz)
  
  # fill in the first row of alp
  alp[1, ] = w * B[, x[1]]
  # Recursively compute the remaining rows of alp
  for(t in 2:T){
    tmp = alp[t-1, ] %*% A
    alp[t, ] = tmp * B[, x[t]]
    }
  return(alp)
}

backward.prob = function(x, para){
  # Output the backward probability matrix beta
  # beta: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  beta = matrix(1, T, mz)

  # The last row of beta is all 1.
  # Recursively compute the previous rows of beta
  for(t in (T-1):1){
    tmp = as.matrix(beta[t+1, ] * B[, x[t+1]])  # make tmp a column vector
    beta[t, ] = t(A %*% tmp)
    }
  return(beta)
}

In [9]:
myViterbi = function(x, para){
  # Output: most likely sequence of Z (T-by-1)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  log.A = log(A)
  log.w = log(w)
  log.B = log(B)
  
  # Compute delta (in log-scale)
  delta = matrix(0, T, mz) 
  # fill in the first row of delta
  delta[1, ] = log.w + log.B[, x[1]]
  
  #######################################
  ## YOUR CODE: 
  ## Recursively compute the remaining rows of delta
  #######################################
  
  # Compute the most prob sequence Z
  Z = rep(0, T)
  # start with the last entry of Z
  Z[T] = which.max(delta[T, ])
  
  #######################################
  ## YOUR CODE: 
  ## Recursively compute the remaining entries of Z
  #######################################
  
  return(Z)
}

In [66]:
data = scan("coding4_part2_data.txt")

mz = 2
mx = 3
ini.w = rep(1, mz); ini.w = ini.w / sum(ini.w)
ini.A = matrix(1, 2, 2); ini.A = ini.A / rowSums(ini.A)
ini.B = matrix(1:6, 2, 3); ini.B = ini.B / rowSums(ini.B)
ini.para = list(mz = 2, mx = 3, w = ini.w,
                A = ini.A, B = ini.B)

myout = myBW(data, ini.para, n.iter = 100)
myout.Z = myViterbi(data, myout)
myout.Z[myout.Z==1] = 'A'
myout.Z[myout.Z==2] = 'B'

In [67]:
library(HMM)
hmm0 =initHMM(c("A", "B"), c(1, 2, 3),
              startProbs = ini.w,
              transProbs = ini.A, 
              emissionProbs = ini.B)
Rout = baumWelch(hmm0, data, maxIterations=100, delta=1E-9, pseudoCount=0)
Rout.Z = viterbi(Rout$hmm, data)

In [ ]:
options(digits=8)
options()$digits

In [ ]:
myout$A
Rout$hmm$transProbs

In [ ]:
myout$B
Rout$hmm$emissionProbs

In [ ]:
cbind(Rout.Z, myout.Z)[c(1:10, 180:200), ]
sum(Rout.Z != myout.Z)